# To-Do:

- InputStandardize vs Normalize

- check hyperparam/prior definitions (scaled space vs raw space)

- keep in mind modality of NaN results in emittance

- Try different number of steps along measurement dimension

- inrease dimensionality of tuning space

- fix legend location (only plot on 1 heatmap)

# In this notebook, we fit a gpytorch GP to a simple emittance model with 1 tuning parameter. We use the GP to evaluate the Expected Information Gain toward the result of a grid-scan minimization routine.

In [1]:
import torch
from emitutils import toy_beam_size_squared_nd, fit_gp_model_emittance
from utils import unif_random_sample_domain
from matplotlib import pyplot as plt
from algorithms import GridMinimizeEmittance
from acquisition import ExpectedInformationGain
from botorch.optim import optimize_acqf
import time
from mpl_toolkits.axes_grid1 import make_axes_locatable
import copy

# Suppress warnings

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Enable/Disable CUDA

In [3]:
use_gpu = True
if use_gpu:
    if torch.cuda.is_available():
        print('CUDA is available. Using default GPU.')
        torch.set_default_tensor_type('torch.cuda.DoubleTensor')
    else:
        print('CUDA not available. Using CPU.')
        use_gpu = False  
else:
    print('Using CPU.')

CUDA is available. Using default GPU.


# Algorithm Settings

In [4]:
# domain = torch.tensor([[-2,2], [-65,35]]).double() #the acquisition domain, must have shape = (ndim, 2)
domain = torch.tensor([[-3,1], [-3,1], [-3,1], [-40,60]]).double() #the acquisition domain, must have shape = (ndim, 2)
ndim = domain.shape[0]                               #where domain[i,0] and domain[i,1] represent
                                                        #the lower and upper bounds of the ith input dimension
                                                        #(these same bounds will be applied to the sampled execution paths) 

    

    
n_samples = 10 #number of posterior samples on which to evaluate execution paths
n_steps_tuning_params = 11 #number of steps per dimension in the posterior sample grid scans 
n_steps_measurement_param = 3
squared = True #whether or not to minimize the "emittance squared" (which can be negative according to the model)








# Monte Carlo Settings

In [ ]:

random_acq = True
n_trials = 10
n_iter = 100
n_obs_init = 5 #number of random observations on which to initialize model


# Run trials

In [5]:
trial_data = {}
trial_data['settings'] = {'domain':domain,
                         'ndim':ndim,
                         'n_obs_init': n_obs_init,
                         'n_samples':n_samples,
                         'n_steps_tuning_params':n_steps_tuning_params,
                         'n_steps_measurement_param': n_steps_measurement_param,
                         'n_trials':n_trials,
                         'n_iter':n_iter,
                         'squared':squared,
                         'random_acq':random_acq}

times = []

for trial in range(n_trials):
    torch.manual_seed(trial)

    #build ndim dimensional parabolic target function
    target_func = toy_beam_size_squared_nd


    ##########################################
    #Observe target function n_obs_init times using a uniform sample of the domain
    x_obs = unif_random_sample_domain(n_samples = n_obs_init, domain = domain)

    y_obs = target_func(x_obs)




    #fit model on initial observations
    model = fit_gp_model_emittance(x_obs, y_obs*1.e6)

    
    algo = GridMinimizeEmittance(domain = domain, 
                   n_samples = n_samples, 
                   n_steps_tuning_params = n_steps_tuning_params,
                    n_steps_measurement_param = n_steps_measurement_param,
                    squared = squared)
    
    rng_state = torch.get_rng_state()
    
        
    if random_acq:
        x_next = None
    else:
        acq_fn = ExpectedInformationGain(model = model, algo = algo)
        
        x_next, _ = optimize_acqf(
            acq_function=acq_fn,
            bounds=acq_fn.algo.domain.T,
            q=1,
            num_restarts=20,
            raw_samples=100,
            options={},
            )

    
    iter_data = {}
    iter_data[0] = {'x_obs': x_obs,
                   'y_obs': y_obs,
                    'x_next': x_next,
                   'model':  copy.deepcopy(model),
                   'rng_state': rng_state}
    
    
    for i in range(1, n_iter+1):
        start = time.time()
        print('Iteration', trial*n_iter + i, '/', n_trials*n_iter)
        
        if random_acq:
            x_new = unif_random_sample_domain(n_samples = 1, domain = domain)
        else:
            x_new = x_next
            
        y_new = target_func(x_new)

        x_obs = torch.cat((x_obs, x_new), dim=0)
        y_obs = torch.cat((y_obs, y_new), dim=0)

        model = fit_gp_model_emittance(x_obs, y_obs*1.e6)

        rng_state = torch.get_rng_state()
        

        
        if random_acq:
            x_next = None
        else:
            acq_fn = ExpectedInformationGain(model = model, algo = algo)
            
            x_next, _ = optimize_acqf(
                acq_function=acq_fn,
                bounds=acq_fn.algo.domain.T,
                q=1,
                num_restarts=20,
                raw_samples=100,
                options={},
                )
            
        end = time.time()
        
        this_time = end - start
        
        print('Operation took', this_time, 'seconds.')
        
        times += [this_time]
        
        total_time = sum(times)
        print('Total time elapsed:', total_time, 'seconds.')
        
        total_iters = (trial*n_iter + i)
        avg_time = total_time/total_iters
        print('Estimated time remaining:', avg_time*(n_trials*n_iter - total_iters), 'seconds.')
        
        print('\n')
        
        iter_data[i] = {'x_obs': x_obs,
                   'y_obs': y_obs,
                    'x_next': x_next,
                   'model':  copy.deepcopy(model),
                   'rng_state': rng_state}
        
    trial_data[trial] = iter_data 


Iteration 1 / 1000
Operation took 0.2699410915374756 seconds.
Total time elapsed: 0.2699410915374756 seconds.
Estimated time remaining: 269.6711504459381 seconds.


Iteration 2 / 1000
Operation took 0.3562445640563965 seconds.
Total time elapsed: 0.6261856555938721 seconds.
Estimated time remaining: 312.46664214134216 seconds.


Iteration 3 / 1000
Operation took 0.2543332576751709 seconds.
Total time elapsed: 0.880518913269043 seconds.
Estimated time remaining: 292.6257855097453 seconds.


Iteration 4 / 1000
Operation took 0.3309290409088135 seconds.
Total time elapsed: 1.2114479541778564 seconds.
Estimated time remaining: 301.65054059028625 seconds.


Iteration 5 / 1000
Operation took 0.28499412536621094 seconds.
Total time elapsed: 1.4964420795440674 seconds.
Estimated time remaining: 297.7919738292694 seconds.


Iteration 6 / 1000
Operation took 0.305330753326416 seconds.
Total time elapsed: 1.8017728328704834 seconds.
Estimated time remaining: 298.4936993122101 seconds.


Iteration

Operation took 0.21771907806396484 seconds.
Total time elapsed: 13.577521562576294 seconds.
Estimated time remaining: 252.64839142911572 seconds.


Iteration 52 / 1000
Operation took 0.21756362915039062 seconds.
Total time elapsed: 13.795085191726685 seconds.
Estimated time remaining: 251.4950146491711 seconds.


Iteration 53 / 1000
Operation took 0.21312975883483887 seconds.
Total time elapsed: 14.008214950561523 seconds.
Estimated time remaining: 250.29772751286345 seconds.


Iteration 54 / 1000
Operation took 0.27948904037475586 seconds.
Total time elapsed: 14.28770399093628 seconds.
Estimated time remaining: 250.29940695232816 seconds.


Iteration 55 / 1000
Operation took 0.23346853256225586 seconds.
Total time elapsed: 14.521172523498535 seconds.
Estimated time remaining: 249.50014608556572 seconds.


Iteration 56 / 1000
Operation took 0.25728797912597656 seconds.
Total time elapsed: 14.778460502624512 seconds.
Estimated time remaining: 249.1226199013846 seconds.


Iteration 57 / 

Iteration 101 / 1000
Operation took 0.2378380298614502 seconds.
Total time elapsed: 28.00409173965454 seconds.
Estimated time remaining: 249.26414330643001 seconds.


Iteration 102 / 1000
Operation took 0.3275580406188965 seconds.
Total time elapsed: 28.331649780273438 seconds.
Estimated time remaining: 249.42962257534847 seconds.


Iteration 103 / 1000
Operation took 0.28571128845214844 seconds.
Total time elapsed: 28.617361068725586 seconds.
Estimated time remaining: 249.22109590919274 seconds.


Iteration 104 / 1000
Operation took 0.32489776611328125 seconds.
Total time elapsed: 28.942258834838867 seconds.
Estimated time remaining: 249.34869150015027 seconds.


Iteration 105 / 1000
Operation took 0.28149867057800293 seconds.
Total time elapsed: 29.22375750541687 seconds.
Estimated time remaining: 249.09774254617236 seconds.


Iteration 106 / 1000
Operation took 0.17440199851989746 seconds.
Total time elapsed: 29.398159503936768 seconds.
Estimated time remaining: 247.94296789169312 s

Operation took 0.1755540370941162 seconds.
Total time elapsed: 42.52170705795288 seconds.
Estimated time remaining: 240.95633999506632 seconds.


Iteration 151 / 1000
Operation took 0.1824202537536621 seconds.
Total time elapsed: 42.70412731170654 seconds.
Estimated time remaining: 240.1046628320454 seconds.


Iteration 152 / 1000
Operation took 0.1805431842803955 seconds.
Total time elapsed: 42.88467049598694 seconds.
Estimated time remaining: 239.25131960919026 seconds.


Iteration 153 / 1000
Operation took 0.18778061866760254 seconds.
Total time elapsed: 43.07245111465454 seconds.
Estimated time remaining: 238.44683721642087 seconds.


Iteration 154 / 1000
Operation took 0.20781421661376953 seconds.
Total time elapsed: 43.28026533126831 seconds.
Estimated time remaining: 237.76041863800643 seconds.


Iteration 155 / 1000
Operation took 0.18883752822875977 seconds.
Total time elapsed: 43.46910285949707 seconds.
Estimated time remaining: 236.97672204048402 seconds.


Iteration 156 / 1

Operation took 0.29924654960632324 seconds.
Total time elapsed: 55.91680121421814 seconds.
Estimated time remaining: 225.07214961099865 seconds.


Iteration 200 / 1000
Operation took 0.32558226585388184 seconds.
Total time elapsed: 56.24238348007202 seconds.
Estimated time remaining: 224.9695339202881 seconds.


Iteration 201 / 1000
Operation took 0.2334139347076416 seconds.
Total time elapsed: 56.47579741477966 seconds.
Estimated time remaining: 224.49831907666143 seconds.


Iteration 202 / 1000
Operation took 0.25573229789733887 seconds.
Total time elapsed: 56.731529712677 seconds.
Estimated time remaining: 224.1176272807735 seconds.


Iteration 203 / 1000
Operation took 0.3123924732208252 seconds.
Total time elapsed: 57.04392218589783 seconds.
Estimated time remaining: 223.96062060177618 seconds.


Iteration 204 / 1000
Operation took 0.2563028335571289 seconds.
Total time elapsed: 57.300225019454956 seconds.
Estimated time remaining: 223.5832309582654 seconds.


Iteration 205 / 1000

...

Operation took 0.34841179847717285 seconds.
Total time elapsed: 227.71464109420776 seconds.
Estimated time remaining: 59.44127453531022 seconds.


Iteration 794 / 1000
Operation took 0.34499692916870117 seconds.
Total time elapsed: 228.05963802337646 seconds.
Estimated time remaining: 59.16912523024628 seconds.


Iteration 795 / 1000
Operation took 0.3285865783691406 seconds.
Total time elapsed: 228.3882246017456 seconds.
Estimated time remaining: 58.89256106082748 seconds.


Iteration 796 / 1000
Operation took 0.3533036708831787 seconds.
Total time elapsed: 228.74152827262878 seconds.
Estimated time remaining: 58.62220071308577 seconds.


Iteration 797 / 1000
Operation took 0.32788705825805664 seconds.
Total time elapsed: 229.06941533088684 seconds.
Estimated time remaining: 58.34515848452952 seconds.


Iteration 798 / 1000
Operation took 0.3445138931274414 seconds.
Total time elapsed: 229.41392922401428 seconds.
Estimated time remaining: 58.07219762312141 seconds.


Iteration 799 / 1

Operation took 0.23530936241149902 seconds.
Total time elapsed: 245.74989700317383 seconds.
Estimated time remaining: 46.11458874881409 seconds.


Iteration 843 / 1000
Operation took 0.22969508171081543 seconds.
Total time elapsed: 245.97959208488464 seconds.
Estimated time remaining: 45.811145856852775 seconds.


Iteration 844 / 1000
Operation took 0.2363908290863037 seconds.
Total time elapsed: 246.21598291397095 seconds.
Estimated time remaining: 45.50911532533112 seconds.


Iteration 845 / 1000
Operation took 0.2736387252807617 seconds.
Total time elapsed: 246.4896216392517 seconds.
Estimated time remaining: 45.21407260838345 seconds.


Iteration 846 / 1000
Operation took 0.23672819137573242 seconds.
Total time elapsed: 246.72634983062744 seconds.
Estimated time remaining: 44.912361553092936 seconds.


Iteration 847 / 1000
Operation took 0.23374724388122559 seconds.
Total time elapsed: 246.96009707450867 seconds.
Estimated time remaining: 44.61026546918516 seconds.


Iteration 848 

Operation took 0.39846014976501465 seconds.
Total time elapsed: 260.5487759113312 seconds.
Estimated time remaining: 31.874092676021434 seconds.


Iteration 892 / 1000
Operation took 0.428997278213501 seconds.
Total time elapsed: 260.9777731895447 seconds.
Estimated time remaining: 31.59820572250093 seconds.


Iteration 893 / 1000
Operation took 0.3191089630126953 seconds.
Total time elapsed: 261.2968821525574 seconds.
Estimated time remaining: 31.30880894773084 seconds.


Iteration 894 / 1000
Operation took 0.33652544021606445 seconds.
Total time elapsed: 261.63340759277344 seconds.
Estimated time remaining: 31.021410743662173 seconds.


Iteration 895 / 1000
Operation took 0.34394192695617676 seconds.
Total time elapsed: 261.9773495197296 seconds.
Estimated time remaining: 30.734772848683363 seconds.


Iteration 896 / 1000
Operation took 0.3324882984161377 seconds.
Total time elapsed: 262.30983781814575 seconds.
Estimated time remaining: 30.446677603891917 seconds.


Iteration 897 / 1

Operation took 0.22298502922058105 seconds.
Total time elapsed: 275.3173336982727 seconds.
Estimated time remaining: 17.262192017213696 seconds.


Iteration 942 / 1000
Operation took 0.2089083194732666 seconds.
Total time elapsed: 275.526242017746 seconds.
Estimated time remaining: 16.964460761177566 seconds.


Iteration 943 / 1000
Operation took 0.19715189933776855 seconds.
Total time elapsed: 275.72339391708374 seconds.
Estimated time remaining: 16.66620726752256 seconds.


Iteration 944 / 1000
Operation took 0.21648311614990234 seconds.
Total time elapsed: 275.93987703323364 seconds.
Estimated time remaining: 16.36931473925962 seconds.


Iteration 945 / 1000
Operation took 0.21031570434570312 seconds.
Total time elapsed: 276.15019273757935 seconds.
Estimated time remaining: 16.072233439753294 seconds.


Iteration 946 / 1000
Operation took 0.2430589199066162 seconds.
Total time elapsed: 276.39325165748596 seconds.
Estimated time remaining: 15.777204640067911 seconds.


Iteration 947 

Operation took 0.37613940238952637 seconds.
Total time elapsed: 290.8131911754608 seconds.
Estimated time remaining: 2.6410885172342553 seconds.


Iteration 992 / 1000
Operation took 0.37722206115722656 seconds.
Total time elapsed: 291.19041323661804 seconds.
Estimated time remaining: 2.3483097841662746 seconds.


Iteration 993 / 1000
Operation took 0.3723001480102539 seconds.
Total time elapsed: 291.5627133846283 seconds.
Estimated time remaining: 2.055326277635849 seconds.


Iteration 994 / 1000
Operation took 0.3721041679382324 seconds.
Total time elapsed: 291.9348175525665 seconds.
Estimated time remaining: 1.7621819972991943 seconds.


Iteration 995 / 1000
Operation took 0.36693358421325684 seconds.
Total time elapsed: 292.3017511367798 seconds.
Estimated time remaining: 1.468853020787838 seconds.


Iteration 996 / 1000
Operation took 0.35167789459228516 seconds.
Total time elapsed: 292.65342903137207 seconds.
Estimated time remaining: 1.1753149760296067 seconds.


Iteration 997 /

In [6]:
import dill
with open('MC-Emittance-NonPhys-Random-4d-Results.pkl', 'wb') as f:
    dill.dump(trial_data, f)